In [1]:
import numpy as np
import matplotlib.pyplot as plt

def NewtonianForce(mass1,mass2, r):
    return (6.67408*10**-11)*mass1*mass2/r/r;

In [2]:
def euler(h,t, x,f):
    return (f(t+h,x+h)-f(t,h)/2./h)

In [3]:
def RK4(h,t,x,y,z,f): #not a finite difference so no step in y
    k1= h*f(t,x,y,z)
    k2=h*f(t+h/2.,x,y,z)
    k3=h*f(t+h/2.,x,y,z)
    k4=h*f(t+h,x,y,z)
    return t+h, f(t,x,y,z)+1/6.*(k1+2.*k2+2.*k3+k4)



In [4]:
def polynomial4(t,x,a,b,c,d,e):
    p=[]
    for xi in x:
        pi=a*xi**4+b*xi**3+c*xi**2+d*xi+e
        #print(pi)
        p.append(pi)
    return np.array(p)
    

def polymaker(t,x,y,z,a,b,c,d,e):
    def poly(t,x,y,z):
        return polynomial4(t,x,a,b,c,d,e)
    return poly
        
        

In [5]:
import numpy as np
t=0.
x=0.01*np.arange(10.)
y=0.01*np.arange(10.)
z=0.01*np.arange(10.)
print(t,x)
polyfn=polymaker(t,x,y,z,1.,0.,0.,0.,0.)

0.0 [0.   0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09]


In [6]:
#import matplotlib.pyplot as plt
#plt.plot(t,polyfn(t,x))

In [7]:
outp=polyfn(t,x,y,z) #this is correct to order of magnitude
print(outp)

[0.000e+00 1.000e-08 1.600e-07 8.100e-07 2.560e-06 6.250e-06 1.296e-05
 2.401e-05 4.096e-05 6.561e-05]


In [8]:
from bokeh.plotting import figure, output_notebook, show
# create a new plot with a title and axis labels

output_notebook()
p = figure(title="Polynomial function (x^4)", x_axis_label='x', y_axis_label='y')

# add a line renderer with legend and line thickness
p.line(x, y, legend="Quartic polynomial", line_width=2)

# show the results
show(p)

Loading BokehJS ...

In [9]:
rk4polyout=RK4(.01,t,x,y,z,polyfn)
print(outp+x)

[0.         0.01000001 0.02000016 0.03000081 0.04000256 0.05000625
 0.06001296 0.07002401 0.08004096 0.09006561]


In [10]:
print(rk4polyout) 
#There are multiple items in the y output, which is confusing. trace this over time and plot an evolution and compare to exact solution, which I have analytically written down. steve points out truncation error
print(((rk4polyout[1]-x-outp)/x))
#order4 polynomial error (comparable in size to polyfn itself, relative error with it is one)

(0.01, array([0.00000e+00, 1.01000e-08, 1.61600e-07, 8.18100e-07, 2.58560e-06,
       6.31250e-06, 1.30896e-05, 2.42501e-05, 4.13696e-05, 6.62661e-05]))
[        nan -0.99999999 -0.99999992 -0.99999973 -0.99999936 -0.99999875
 -0.99999784 -0.99999657 -0.99999488 -0.99999271]


/Users/sdorsher/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


In [11]:
def invpolyint(t,x):
    p=[]
    for ti in t:
        xi=-3*t**(-1./3.)
        p.append(xi)
    return np.array(p)
    



In [12]:
#print(invpolyint(t,x)) #need an entire run to invert this because you need a history of a timeseries accumulated

In [13]:
def analyticalpolyint(t,x):
    p=[]
    for xi in x:
        ti = -1./3.*xi**(-3.)
        p.append(ti)
    return np.array(p)

In [14]:
print(x)

[0.   0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09]


In [15]:
#print(analyticalpolyint(t,x))

In [16]:
def invertIMF(luminosity,lsun,masssun):
    M=[]
    for L in luminosity: 
        if (L<(.43)**4)*lsun:
            x=((L/lsun/0.23)**(1./2.3))*masssun
            #print(x,1)
            M.append(x)
        elif (L<16*lsun):
            x=(((L/lsun)**(0.25))*masssun)
            #print(L,lsun,masssun,x,2)
            M.append(x)
        elif (L<(((20**(3.5))*1.4)*masssun)):
            x=(((L/lsun/1.4)**(1./3.5))*masssun)
            #print(x,3)
            M.append(x)
        else:
            x=((L/lsun/32000.)*masssun)
            #print(x,4)
            M.append(x)
    return np.array(M)
    

In [17]:
def InitialData():
    random.seed(a=9001)
    
    #initially use an in plane orbit with random starting locations relative to the x axis
    phi=np.random.uniform(0,1,2)
    orbitangle=np.zeros(2)
    #orbitalradius=np.random.uniform(.1,50,2)
    orbitalradius=np.ones(2)
    #start with circular orbits
    #eccentricity=np.random.uniform(0.,.1,2)
    eccentricity=np.zeros(2)
    #magnitude=np.random.uniform(-20,-30,2) #absolute not apparent maginutde
    #magsun=-26.832
    masssun=1.989*10**30
    lsun=3.828*10**26
    massearth=5.9722*10**24
    #luminosity=lsun*10**(0.4*(magnitude-magsun))
    #masses= invertIMF(luminosity,lsun,masssun) #Initial mass function for Main Sequence
    #masses=np.random.uniform(.7,5.) #replace with IMF
    masses=np.array([masssun,massearth])

    return phi,orbitangle,orbitalradius,eccentricity, masses



In [18]:
import random
initdat=InitialData()

In [19]:
print(initdat)

(array([0.98839704, 0.21768559]), array([0., 0.]), array([1., 1.]), array([0., 0.]), array([1.9890e+30, 5.9722e+24]))


In [20]:
def getxyuv(initdat):
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    #print(orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=149597870700
    Gconstant=6.408*10**-11
    x0=orbitalradius*np.cos(phi)*metersperAU
    y0=orbitalradius*np.sin(phi)*metersperAU
    z0=np.zeros(2)
    

    
    #start at perihelion for both (eliptical, doesn't generalize to three body)
    #actually start with circular orbit
    ux0=np.zeros(2)*149597870700
    #centrepital force balances gravitational force
    reducedmass=np.zeros(2)
    print(masses)
    for i in np.arange(2):
        j=(i+1)%2 #reverse masses
        reducedmass[i]=masses[i]*masses[j]/np.sum(masses)
    print(reducedmass)
    rphys=orbitalradius*metersperAU*reducedmass/np.sum(masses)
    print(rphys)
    F=(Gconstant*reducedmass**2/rphys**2)
    print(F)
    #centF=reducedmass*v**2/rphys
    #centF=accel
    v=np.sqrt(Gconstant*reducedmass/rphys)
    print(v)
    uy0=v #initial data in y only
    #evolve in plane only
    #there is a units problem that needs to be fixed
    #velocity initial conditions are not trivial. 
    uz0=np.zeros(2)
    
    #circular orbit   #a=omega^2 * r #v=omega*r #omega=v/r
    omega=v/rphys
    print(omega)
    omegatrue=np.mean(omega) #shouldaccount for numerical effects
    print(omegatrue)
    a=omegatrue**2*rphys
    ax0=a*np.cos(phi)
    ay0=a*np.sin(phi)
    az0=np.zeros(2)
    
    
    return reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0

In [21]:
xyuva=getxyuv(initdat)
print(xyuva)#In SI units
print(xyuva[1][0])
print(xyuva[0][0]/xyuva[0][1])

[1.9890e+30 5.9722e+24]
[5.97218207e+24 5.97218207e+24]
[449182.02019948 449182.02019948]
[1.13277552e+28 1.13277552e+28]
[29188.82340899 29188.82340899]
[0.06498217 0.06498217]
0.06498217225174459
(array([5.97218207e+24, 5.97218207e+24]), array([8.22832083e+10, 1.46067347e+11]), array([1.24935970e+11, 3.23087135e+10]), array([0., 0.]), array([0., 0.]), array([29188.82340899, 29188.82340899]), array([0., 0.]), array([1043.26975964, 1851.9896034 ]), array([1584.06462081,  409.64255538]), array([0., 0.]))
82283208312.60228
1.0


In [22]:
def timestep(step,t,dt,reducedmass,xi,yi,zi, vxi, vyi, vzi, axi, ayi, azi):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vx))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vy))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=axi
    ayii=ayi
    azii=azi
    
    for m in np.arange(len(x)):
        #m represents choices of mass
        i=step
        
        xii[m] = xi[m] + dt*vxi[m]
        #print(xii)
        vxii[m] = vxi[m] + dt*axi[m]
        #print(vxii)
        yii[m]= yi[m] + dt*vyi[m]
        vyii[m] = vyi[m] + dt*ayi[m]
        zii[m]= zi[m] + dt*vzi[m]
        vzii[m] = vzi[m] + dt*azi[m]
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
    
    
    Gconstant=6.408*10**-11
    for k in np.arange(len(rii)):
        for j in np.arange(len(rii)):
            if j!=k:
                rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                axii[j]+=Gconstant*reducedmass[k]*(xi[j]  - xi[k])/rreljk**3
                ayii[j]+=Gconstant*reducedmass[k]*(yi[j]  - yi[k])/rreljk**3
                azii[j]+=Gconstant*reducedmass[k]*(zi[j]  - zi[k])/rreljk**3
    #print(xii)
    return reducedmass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii
                    

In [23]:
dt=0.01*31556926 #seconds per year
numsteps=5
mass,x,y,z,vx,vy,vz,ax,ay,az=xyuva
for i in np.arange(1,numsteps):
    t=0.+i*numsteps*dt
    mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(i,t,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
    print(x,y,vx,vy,ax,ay)
    #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
    #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
    #print(ax)
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(2,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(3,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(4,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 

[8.22832083e+10 1.46067347e+11] [1.34147065e+11 4.15198089e+10] [3.29223866e+08 5.84430989e+08] [4.99911289e+08 1.29299787e+08] [1043.26975962 1851.98960341] [1584.06462083  409.64255535]
[1.03975215e+14 1.84574522e+14] [1.57890783e+14 4.08445579e+13] [6.58447732e+08 1.16886198e+09] [9.99793389e+08 2.58570385e+08] [1043.2697596  1851.98960343] [1584.06462086  409.64255533]
[3.11761079e+14 5.53431431e+14] [4.73394843e+14 1.22441423e+14] [9.87671598e+08 1.75329297e+09] [1.49967549e+09 3.87840983e+08] [1043.2697596  1851.98960343] [1584.06462086  409.64255533]
[6.23439874e+14 1.10671680e+15] [9.46646327e+14 2.44832115e+14] [1.31689546e+09 2.33772395e+09] [1.99955759e+09 5.17111581e+08] [1043.2697596  1851.98960343] [1584.06462086  409.64255533]


In [24]:
def timestepRK4(step,t,dt,reducedmass,xi,yi,zi, vxi, vyi, vzi, axi, ayi, azi):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vx))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vy))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=axi
    ayii=ayi
    azii=azi
    
    
    tnew,ynew, intval=RK4(h,t,y,f)
    for m in np.arange(len(x)):
        #m represents choices of mass
        i=step
        
        tnew,ynew,intvalx=RK4(h,t,xi[m],vxif)
        xii[m] = intvalx
        #print(xii)
        tnew,ynew,intvalvx=RK4(h,t,vxyi[m], axif)
        vxii[m] = vxi[m] + dt*axi[m]
        #print(vxii)
        yii[m]= yi[m] + dt*vyi[m]
        vyii[m] = vyi[m] + dt*ayi[m]
        zii[m]= zi[m] + dt*vzi[m]
        vzii[m] = vzi[m] + dt*azi[m]
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
    
    
    Gconstant=6.408*10**-11
    for k in np.arange(len(rii)):
        for j in np.arange(len(rii)):
            if j!=k:
                rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                axii[j]+=Gconstant*reducedmass[k]*(xi[j]  - xi[k])/rreljk**3
                ayii[j]+=Gconstant*reducedmass[k]*(yi[j]  - yi[k])/rreljk**3
                azii[j]+=Gconstant*reducedmass[k]*(zi[j]  - zi[k])/rreljk**3
    #print(xii)
    return reducedmass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii

In [35]:
class OrbitDiffEq:
    def __init__(self,reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0):
        self.reducedmass=reducedmass
        self.xi=x0
        self.yi=y0
        self.zi=z0
        self.vxi=ux0
        self.vyi=uy0
        self.vzi=uz0
        self.axi=ax0
        self.ayi=ay0
        self.azi=az0
    def dxidt(t,xi, yi, zi):
        return self.vxi
    def dxidt(t,xi, yi, zi):
        return self.vyi
    def dxidt(t,xi, yi, zi):
        return self.vzi
    def dvxidt(t, vxi, vyi, vzi):
        #return axi[m]
        axii=self.axi
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                    axii[j]+=Gconstant*reducedmass[k]*(xi[j]  - xi[k])/rreljk**3
        self.axi=axii
        return axii
    def dvyidt(t, vxi, vyi, vzi):
        #return axi[m]
        ayii=self.ayi
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                    ayii[j]+=Gconstant*reducedmass[k]*(xi[j]  - xi[k])/rreljk**3
        self.ayi=ayii
        return ayii
    def dvzidt(t, vxi, vyi, vzi):
        #return axi[m]
        azii=self.azi
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
        Gconstant=6.408*10**-11
        for k in np.arange(len(rii)):
            for j in np.arange(len(rii)):
                if j!=k:
                    rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                    azii[j]+=Gconstant*reducedmass[k]*(xi[j]  - xi[k])/rreljk**3
        self.azi=azii
        return azii
    def update(reducedmass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii):
        self.xi=xii
        self.yi=yii
        self.zi=zii
        self.vxi=vxii
        self.vyi=vyii
        self.vzi=vzii
        self.axi=axii
        self.ayi=ayii
        self.azi=azii
    

In [ ]:
def RK4(h,t,x,y,z,f): #not a finite difference so no step in y
    k1= h*f(t,x,y,z)
    k2=h*f(t+h/2.,x,y,z)
    k3=h*f(t+h/2.,x,y,z)
    k4=h*f(t+h,x,y,z)
    return t+h, f(t,x,y,z)+1/6.*(k1+2.*k2+2.*k3+k4)




In [34]:
reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0=xyuva
ODE= OrbitDiffEq(reducedmass,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0)

In [36]:
def timestepRK4(step,t,dt,reducedmass,ODE):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vx))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vy))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=axi
    ayii=ayi
    azii=azi
    
    
    tnew,ynew, intval=RK4(h,t,y,f)
    for m in np.arange(len(x)):
        #m represents choices of mass
        i=step
        
        tnew,intvalx=RK4(h,t,xi[m],ODE.dxidt)
        xii[m] = intvalx
        tnew,intvalvx=RK4(h,t,vxi[m], ODE.dvxidt)
        vxi[m]=intvalvx
        tnew,intvaly=RK4(h,t,yi[m],ODE.dyidt)
        yii[m] = intvaly
        tnew,intvalvy=RK4(h,t,vyi[m], ODE.dvyidt)
        vyi[m]=intvalvy
        tnew,intvalz=RK4(h,t,xi[m],ODE.dzidt)
        zii[m] = intvalz
        tnew,intvalvz=RK4(h,t,vzi[m], ODE.dvzidt)
        vzi[m]=intvalvz
        #vxii[m] = vxi[m] + dt*axi[m]
        #print(vxii)
        #yii[m]= yi[m] + dt*vyi[m]
        #vyii[m] = vyi[m] + dt*ayi[m]
        #zii[m]= zi[m] + dt*vzi[m]
        #vzii[m] = vzi[m] + dt*azi[m]
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
    
    
    Gconstant=6.408*10**-11
    for k in np.arange(len(rii)):
        for j in np.arange(len(rii)):
            if j!=k:
                rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                axii[j]+=Gconstant*reducedmass[k]*(xi[j]  - xi[k])/rreljk**3
                ayii[j]+=Gconstant*reducedmass[k]*(yi[j]  - yi[k])/rreljk**3
                azii[j]+=Gconstant*reducedmass[k]*(zi[j]  - zi[k])/rreljk**3
    #print(xii)
    ODE.update(reducedmass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii)
    return reducedmass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii